<a href="https://colab.research.google.com/github/ev1025/GA_LOG_ANALYST/blob/main/Ga%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
from pandas import json_normalize
import json
import os
import csv
import re
import ast
import copy
from tqdm.notebook import tqdm


In [ ]:
# train_v2가 있는 경로로 이동
# os.chdir('D:\\train_v2.csv')

**데이터 가져오기**

- 데이터 대용량(25GB)이슈로 인하여 read_csv의 chunksize를 사용하여 30만개의 데이터만 추출
- 확인결과 2016년 10월 ~ 2018년 4월 데이터 추출
- 데이터 용량이 4GB로 축소하였으나, 태블로 사용시 권장 용량이 4GB미만인 점과 1년의 추세를 보고자 2017년 자료로 사용

In [ ]:
# # address에 chunk 저장
# address = pd.read_csv('train_v2.csv', chunksize=10000,
#                        dtype={'fullVisitorId':'str',('date','fulVisitorid', 'visitId', 'visitNumber', 'visitStartTime'):'int'})
# # 빈프레임 생성
# train_data = pd.DataFrame()

In [ ]:
# # 1만개씩 분할하여 빈 프레임에 30만개 수집
# i = 0
# for row, chunk in enumerate(address):
#     train_data = pd.concat([train_data, chunk])
#     print(f'{row*10000}개')
#     i+=1
#     if i == 30:
#         break

In [ ]:
# # 2017년 GA쇼핑몰데이터만 저장
# ga_2017 = train_data[train_data['date'].astype('str').str.contains('2017')]
# ga_2017.to_csv('2017_ga_data.csv', index=False, lineterminator='\n')

In [ ]:
# df = pd.read_csv('2017_ga_data.csv', 
#                 dtype={'fullVisitorId': 'str'})

In [ ]:
# df2 = df.copy()

- 'hits' 컬럼에 read_csv의 converter가 적용되지 않는 이슈가 발생
- 원인을 찾지 못해서 따로 json.loads 진행해주기로 결정
- ' '로 묶인 문자는 json에서 사용할 수 없다는 오류 발생하여 '를 "로 변경
- Bool형식을 인식하지 못해서 True => "True" , False => "False"로 변경
- ' 을 "로 바꾸는 과정에서 제품명에 Men's와 같은 문자가 포함되어 있어서 문자열이 끊어지는 현상 발생
- 결국 ast.literal_eval() 함수를 apply하여 json형식으로 변경

In [ ]:
## 각 컬럼의 json을 활성화(str=>json)
# JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource','hits']

# for i in JSON_COLUMNS:
#     df2[i] = df2[i].apply(lambda x : ast.literal_eval(x))
#     print(f'{i} 끝')

In [ ]:
# df2.to_csv('all_convert.csv',index=False)

In [ ]:
# # json형태의 컬럼들을  key : value 형태로 컬럼화
# json_to_col = ['device','geoNetwork', 'totals', 'trafficSource']
# for column in json_to_col:
#         column_as_df = json_normalize(df2[column])
#         column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
#         df2 = df2.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
# df2.shape

In [ ]:
os.chdir('C:\\Users\\ENVY\\Desktop')
df2 = pd.read_csv('json_convert.csv')

C:\Users\ENVY\AppData\Local\Temp\ipykernel_7224\3722996920.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('json_convert.csv')


- customDimensions와 hits컬럼은 이중 json이라 따로 정리

In [ ]:
# 이중 dict형태라서 따로 정리
tqdm.pandas()
df2['customDimensions'] = df2['customDimensions'].progress_apply(lambda x : ast.literal_eval(x))

  0%|          | 0/156860 [00:00<?, ?it/s]

In [ ]:
custom = json_normalize(df2['customDimensions'])
cus_col = json_normalize(custom[0])
cus_col.columns = [f'{"customDimensions"}.{subcolumn}' for subcolumn in cus_col.columns]
df2 = df2.drop(['customDimensions'],axis=1).merge(cus_col,right_index=True, left_index=True)

In [ ]:
# 조회한 횟수 만큼 컬럼이 생김
hit = df2['hits'].progress_apply(lambda x : ast.literal_eval(x))

  0%|          | 0/156860 [00:00<?, ?it/s]

In [ ]:
# 빈 컬럼과 데이터 컬럼 생성(hit수 만큼 컬럼이 생김)
emp_data = pd.DataFrame()
hits_col = json_normalize(hit)
hits_col.head()

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,"{'hitNumber': '1', 'time': '0', 'hour': '17', ...",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,"{'hitNumber': '1', 'time': '0', 'hour': '10', ...","{'hitNumber': '2', 'time': '27844', 'hour': '1...",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,"{'hitNumber': '1', 'time': '0', 'hour': '17', ...","{'hitNumber': '2', 'time': '38153', 'hour': '1...",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,"{'hitNumber': '1', 'time': '0', 'hour': '9', '...","{'hitNumber': '2', 'time': '1087', 'hour': '9'...",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,"{'hitNumber': '1', 'time': '0', 'hour': '14', ...","{'hitNumber': '2', 'time': '52349', 'hour': '1...",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
# # hitsnum별로 옆으로 이어붙이기(필요할까?)
# for i in tqdm(range(len(hits_col.columns))):
#     emp_data = pd.concat([emp_data, json_normalize(hits_col[i])], axis=1)

# emp_data.head()

  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
# 일단 hitNumber1에 대해서만 분석하기로 결정
emp_data = pd.concat([emp_data, json_normalize(hits_col[0])],axis=1)
df2 = df2.drop(['hits'],axis=1).merge(emp_data,right_index=True, left_index=True)
df2.to_csv('hits_last.csv',index=False)

In [ ]:
df2.head()

,channelGrouping,date,fullVisitorId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserVersion,device.browserSize,...,latencyTracking.pageLoadTime,latencyTracking.pageDownloadTime,latencyTracking.redirectionTime,latencyTracking.speedMetricsSample,latencyTracking.domainLookupTime,latencyTracking.serverConnectionTime,latencyTracking.serverResponseTime,latencyTracking.domLatencyMetricsSample,latencyTracking.domInteractiveTime,latencyTracking.domContentLoadedTime
0,Organic Search,20171016,3162355547410993243,Not Socially Engaged,1508198450,1,1508198450,Firefox,not available in demo dataset,not available in demo dataset,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Referral,20171016,8934116514970143966,Not Socially Engaged,1508176307,6,1508176307,Chrome,not available in demo dataset,not available in demo dataset,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Direct,20171016,7992466427990357681,Not Socially Engaged,1508201613,1,1508201613,Chrome,not available in demo dataset,not available in demo dataset,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Organic Search,20171016,9075655783635761930,Not Socially Engaged,1508169851,1,1508169851,Chrome,not available in demo dataset,not available in demo dataset,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Organic Search,20171016,6960673291025684308,Not Socially Engaged,1508190552,1,1508190552,Chrome,not available in demo dataset,not available in demo dataset,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
